In [ ]:
import os
import sys

os.environ['CADENCE_SIMS'] = '/mnt/md0/sn-sims/'
sys.path.insert(0, '../')


# Cadence Effects

This notebook demonstrates the simulation of normal Type Ia Supernova (SN Ia) light-curves using realistic cadences and atmospheric variability expected from LSST. To achieve this we proceed as follows:

1. Use data from the PLaSTICC simulations to establish the cadence, light-curve parameters, and location of SNe observed by LSST.
2. Apply time variable PWV transmission effects to a simulated light-curve
3. Simulate and fit a handful of light-curves for a single cadence and analyze the results.



In [ ]:
import numpy as np
import sncosmo
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.table import Table
from matplotlib import pyplot as plt
from pwv_kpno.gps_pwv import GPSReceiver

from snat_sim import filters, plasticc,  sn_magnitudes, modeling, constants, weather


In [ ]:
filters.register_lsst_filters(force=True)

plt.rcParams['figure.dpi'] = 100  # Enable HDPI
print('Simulation data directory:', plasticc.plasticc_simulations_directory)


## The PLaSTICC Data

Instead of evaluating different cadences from scratch, we use light-curves from the PLaSTICC simulations. First we check what cadence simulations are available on the notebook's host server.


In [ ]:
plasticc.get_available_cadences()


Simulated light-curves are written in the SNANA file format and are distributed across multiple files. We load a light-curve from one of these files and demonstrate the data model below. Each cadence includes simulations run with multiple supernova models. In this notebook we only need simulations for normal SNe (Model 11).


In [ ]:
demo_cadence = 'alt_sched'
demo_cadence_header_files = plasticc.get_model_headers(demo_cadence, 11)

print('Available cadence files:', len(demo_cadence_header_files))
print('Available Light-curves: ', plasticc.count_light_curves(demo_cadence, model=11))
    

In [ ]:
demo_header_path = demo_cadence_header_files[0]
plasticc_lc = next(plasticc.iter_lc_for_header(demo_header_path, verbose=False))


In [ ]:
plasticc_lc.meta


In [ ]:
plasticc_lc


Here we reformat the data to be compatible with `sncosmo` so we can easily visualize the light-curve.


In [ ]:
formatted_lc = plasticc.format_plasticc_sncosmo(plasticc_lc)


In [ ]:
sncosmo.plot_lc(formatted_lc);


## Simulating Light-Curves

Since we need to add in our own atmospheric variability, the pre-tabulated flux values above are of limited use. Instead, we use the PLaSTICC light-curves to establish the cadence and model parameters for each simulated SN. This information is then used to simulate our own light-curves with `sncosmo`.


**Note:** See Issue 8 (https://github.com/LSSTDESC/SN-PWV/issues/8) for caveats about the following cell.

In [ ]:
model_for_sim = sncosmo.Model('salt2-extended')
duplicated_lc = plasticc.duplicate_plasticc_sncosmo(plasticc_lc, model_for_sim, gain=20, skynr=100)


In [ ]:
duplicated_lc.meta


In [ ]:
sncosmo.plot_lc(duplicated_lc);


In [ ]:
duplicated_lc


The `sncosmo` package doesn't have a clearly defined approach to adding time variable propagation effects, so we use the custom `snat_sim.Model` class which is similar to the `sncosmo.Model` class but overloads the underlying flux calculation by adding PWV transmission effects. Note that in this approach the `t0` parameter for each light-curve is now in units of MJD (i.e., it must be the same units as the interpolation function).


In [ ]:
def plot_variable_pwv_model(model_with_pwv, phase=0, params=dict()):
    """Overplot a sncosmo model with and without temporally variable PWV
    
    Args:
        source (str, Source): sncosmo source to plot
        phase        (float): Phase of the supernova to plot
        params        (dict): Non-PWV related parameters for the model
    """
    
    wave = np.arange(3000, 12000)
    time = phase + params['t0']

    model_without_pwv = sncosmo.Model(model_with_pwv.source)
    model_without_pwv.update(params)
    flux_without_pwv = model_without_pwv.flux(time, wave)

    model_with_pwv.update(params)
    flux_with_pwv = model_with_pwv.flux(time, wave)

    fig, ax = plt.subplots(1, 1, figsize=(6, 3))
    ax.plot(wave, flux_without_pwv, label='Base Model', color='C1')
    ax.plot(wave, flux_with_pwv, label='Model with PWV', color='C0')
    ax.set_title('Simulated Flux')
    ax.set_ylabel('Flux')
    ax.legend()
    ax.set_xlabel('Wavelength (A)')
    ax.set_xlim(min(wave), max(wave))

    plt.tight_layout()


In [ ]:
ctio = GPSReceiver('CTIO', data_cuts={'PWV': [(0, 25)]})
ctio.download_available_data(year=range(2012, 2018))

pwv_model = weather.build_suominet_model(ctio, 2016, [2017])
pwv_effect_no_airmass = modeling.VariablePWVTrans(pwv_model, scale_airmass=False)
pwv_effect_no_airmass.set(res=5)

demo_model = modeling.Model(
    source='salt2-extended',
    effects=[pwv_effect_no_airmass],
    effect_names=[''],
    effect_frames=['obs']
)

plot_variable_pwv_model(
    demo_model,     
    params = {
        'z': 0.752069652,
        't0': 61900,
        'x0': 3e-06,
        'x1': -1.8,
        'c': -0.1}
)


## Airmass Scaling


In [ ]:
def plot_airmass_validation(cadence, model=11, mjd=0):
    ra = []
    dec = []
    peak = []

    for header_path in plasticc.get_model_headers(cadence, 11):
        header_data = fits.open(header_path)[1].data
        ra.extend(header_data['RA'])
        dec.extend(header_data['DECL'])
        if mjd == 'peak':
            peak.extend(header_data['PEAKMJD'])

    if mjd == 'peak':
        mjd = peak
    
    airmass = modeling.calc_airmass(
        time=mjd,
        ra=ra,
        dec=dec,
        lat=constants.vro_latitude.value,
        lon=constants.vro_longitude.value,
        alt=constants.vro_longitude.value
    )

    
    is_positive_airmass = np.array(airmass) >= 0 
    positive_airmass = airmass[is_positive_airmass]
    
    lsst_coord = SkyCoord(constants.vro_latitude, constants.vro_longitude).galactic
    sn_coord = SkyCoord(ra, dec, unit=u.deg).galactic
    positive_coords = sn_coord[is_positive_airmass]
    negative_coords = sn_coord[~is_positive_airmass]
    
    plt.figure(figsize=(10, 5))
    plt.subplot(111, projection='aitoff')
    plt.grid(True)
    
    scat = plt.scatter(positive_coords.l.wrap_at('180d').radian, positive_coords.b.radian, c=positive_airmass, vmin=1, vmax=8, s=10)
    plt.scatter(negative_coords.l.wrap_at('180d').radian, negative_coords.b.radian, c='lightgrey', label='Over Horizon')
    plt.scatter(lsst_coord.l.wrap_at('180d').radian, lsst_coord.b.radian,color='C1', marker='*', s=100, label='VRO')
    plt.legend(framealpha=1)
    plt.colorbar(scat).set_label('Airmass', rotation=270, labelpad=15)
    

In [ ]:
plot_airmass_validation(demo_cadence)


## Fitting Light-Curves

We create an iterator that extracts cadence data from PLaSTICC light-curves and simulates custom light-curves with time variable PWV effects. We then fit each light-curve and look at the aggregate properties. In order to ensure a successful fit for each light-curve, we apply the following quality cuts:

1. Light-curves must have at least one point with SNR >= 5 in two or more bands
2. So that the simulated data is within the wavelength range of the model, light-curves with a redshift greater than .8 are dropped. 

In [ ]:
def iter_custom_lcs(
        model, cadence, iter_lim=None, gain=20, skynr=100, quality_callback=None, verbose=True):
    """Simulate light-curves for a given PLaSTICC cadence
    
    Args:
        model               (Model): Model to use in the simulations
        cadence               (str): Cadence to use when simulating light-curves
        gain                  (int): Gain to use during simulation
        skynr                 (int): Simulate skynoise by scaling plasticc ``SKY_SIG`` by 1 / skynr
        quality_callback (callable): Skip light-curves if this function returns False
        verbose              (bool): Display a progress bar
    """
    
    # model = copy(model)
    
    # Determine redshift limit of the given model
    u_band_low = sncosmo.get_bandpass('lsst_hardware_u').minwave()
    source_low = model.source.minwave()
    zlim = (u_band_low / source_low) - 1
    
    counter = -1
    iter_lim = float('inf') if iter_lim is None else iter_lim
    for light_curve in plasticc.iter_lc_for_cadence_model(cadence, model=11, verbose=verbose):
        counter += 1
        if counter >= iter_lim:
            break
        
        if light_curve.meta['SIM_REDSHIFT_CMB'] >= zlim:
            continue
        
        model.set(ra=light_curve.meta['RA'], dec=light_curve.meta['DECL'])
        duplicated_lc = plasticc.duplicate_plasticc_sncosmo(light_curve, model, gain=gain, skynr=skynr)
        
        # sncosmo.plot_lc(duplicated_lc)
        # plt.show()

        if quality_callback and not quality_callback(duplicated_lc):
            continue
            

        yield duplicated_lc 
        

In [ ]:
def passes_quality_cuts(light_curve):
    """Return whether light-curve has 2+ two bands each with 1+ data point with SNR > 5
    
    Args:
        light_curve (Table): Astropy table with sncosmo formatted light-curve data
        
    Returns:
        A boolean
    """
    
    if light_curve.meta['z'] > .88:
        return False
    
    light_curve = light_curve.group_by('band')
    
    passed_cuts = []
    for band_lc in light_curve.groups:
        passed_cuts.append((band_lc['flux'] /  band_lc['fluxerr'] > 5).any())
        
    return sum(passed_cuts) >= 2
        

We pause to visually check a light-curves from our iterator. As a simple validation, we simulate light-curves with and without PWV.

In [ ]:
variable_pwv_effect = modeling.VariablePWVTrans(pwv_interpolator)
variable_pwv_effect.set(res=5)

sn_model_with_pwv = modeling.Model(
    source='salt2-extended',
    effects=[variable_pwv_effect],
    effect_names=[''],
    effect_frames=['obs']
)


In [ ]:
l = next(iter_custom_lcs(sn_model_with_pwv, demo_cadence, verbose=False))
sncosmo.plot_lc(l);


In [ ]:
light_curves = iter_custom_lcs(sn_model_with_pwv, demo_cadence, iter_lim=5, quality_callback=passes_quality_cuts)

model_without_pwv = sncosmo.Model('salt2-extended')
fitted_mag, fitted_params = sn_magnitudes.fit_mag(
    model=model_without_pwv, 
    light_curves=light_curves, 
    vparams=['x0', 'x1', 'c'], 
    bands=['lsst_hardware_' + b for b in 'ugrizy'])
